In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import time
import pymongo
from datetime import datetime
import pytz

from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import ApplicationBuilder
from telegram.constants import ParseMode

from bot import config
from bot import database
from bot.app import show_balance_handle

In [ ]:
application = (
    ApplicationBuilder()
    .token(config.telegram_token)
    .build()
)

db = database.Database()

In [ ]:
newsletter_id = "day_pi"

db.create_newsletter(newsletter_id)
already_sent_to_user_ids = set(db.get_newsletter_attribute(newsletter_id, "already_sent_to_user_ids"))

print(f"Already sent to {len(already_sent_to_user_ids)} users")

In [ ]:
user_dicts = list(db.user_collection.find({"username": "karfly"}))
# user_dicts = list(db.user_collection.find({}))
print(f"Found {len(user_dicts)} users")

In [ ]:
for user_dict in user_dicts:        
    if user_dict["_id"] in already_sent_to_user_ids:
        print(f"Skipping {user_dict['_id']}. Already sent before")
        continue
        
    text = "🥧 Happy <b>pi</b> day!\n\n"
    text += "Only today you can buy <b>3,141,592</b> tokens for <s>75$</s> <b>29.99$</b> (60% discount 😱)"
    
    reply_markup = InlineKeyboardMarkup([
        [InlineKeyboardButton("🟣 Buy 3,141,592 tokens", callback_data=f"send_payment||29.99|3141592")],
    ])

    try:        
        await application.bot.send_animation(
            user_dict['chat_id'],
            "./static/pi_day.mp4",
            caption=text,
            parse_mode=ParseMode.HTML,
            reply_markup=reply_markup
            
        )
        print(f"Successfully sent to {user_dict['_id']}")
        db.add_user_to_newsletter(newsletter_id, user_dict["_id"])
        
        time.sleep(7)
    except Exception as e:
        print(f"Failed to send message to {user_dict['_id']}. Reason: {e}")

In [ ]:
D